In [1]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

/home/deniz/anaconda3/envs/NLP/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
len(documents)

1226258

In [3]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [4]:

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

/home/deniz/anaconda3/envs/NLP/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/deniz/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [7]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [8]:

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [10]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [11]:
processed_docs = documents['headline_text'].map(preprocess)

In [12]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [13]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [14]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3570, 1), (3571, 1)]

In [16]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3570 ("compulsori") appears 1 time.
Word 3571 ("ratepay") appears 1 time.


# TF-ID

In [17]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [18]:
corpus_tfidf = tfidf[bow_corpus]

In [19]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5842699484464488),
 (1, 0.38798859072167835),
 (2, 0.5008422243250992),
 (3, 0.5071987254965034)]


# Running LDA using Bag of Words

In [20]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [21]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"death" + 0.025*"charg" + 0.025*"case" + 0.024*"court" + 0.020*"polic" + 0.020*"murder" + 0.015*"alleg" + 0.013*"trial" + 0.012*"arrest" + 0.012*"face"
Topic: 1 
Words: 0.022*"news" + 0.020*"market" + 0.018*"world" + 0.017*"women" + 0.015*"australian" + 0.015*"final" + 0.014*"island" + 0.012*"return" + 0.011*"street" + 0.010*"fall"
Topic: 2 
Words: 0.054*"coronavirus" + 0.035*"covid" + 0.024*"live" + 0.021*"nation" + 0.020*"coast" + 0.016*"restrict" + 0.014*"water" + 0.013*"gold" + 0.011*"plan" + 0.010*"park"
Topic: 3 
Words: 0.038*"sydney" + 0.025*"polic" + 0.021*"crash" + 0.020*"adelaid" + 0.019*"die" + 0.015*"miss" + 0.012*"break" + 0.011*"drug" + 0.011*"driver" + 0.010*"road"
Topic: 4 
Words: 0.037*"year" + 0.031*"melbourn" + 0.022*"open" + 0.021*"canberra" + 0.017*"jail" + 0.015*"work" + 0.014*"high" + 0.014*"life" + 0.013*"interview" + 0.013*"offic"
Topic: 5 
Words: 0.029*"govern" + 0.019*"health" + 0.019*"school" + 0.017*"help" + 0.017*"chang" + 0.015*"fed

# Running LDA using TF-IDF

In [22]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)


In [23]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.027*"trump" + 0.009*"final" + 0.008*"scott" + 0.007*"thursday" + 0.007*"australia" + 0.007*"leagu" + 0.006*"premier" + 0.006*"test" + 0.006*"alan" + 0.005*"grandstand"
Topic: 1 Word: 0.019*"news" + 0.018*"market" + 0.015*"donald" + 0.013*"rural" + 0.012*"coronavirus" + 0.012*"drum" + 0.009*"price" + 0.008*"tuesday" + 0.008*"share" + 0.007*"wednesday"
Topic: 2 Word: 0.008*"australia" + 0.008*"cricket" + 0.007*"juli" + 0.007*"world" + 0.006*"rugbi" + 0.006*"australian" + 0.006*"august" + 0.006*"award" + 0.005*"win" + 0.005*"biden"
Topic: 3 Word: 0.020*"countri" + 0.015*"hour" + 0.011*"weather" + 0.009*"michael" + 0.009*"turnbul" + 0.009*"extend" + 0.007*"brief" + 0.007*"disabl" + 0.007*"octob" + 0.005*"america"
Topic: 4 Word: 0.020*"live" + 0.011*"search" + 0.011*"covid" + 0.010*"andrew" + 0.009*"updat" + 0.009*"financ" + 0.009*"coronavirus" + 0.009*"peter" + 0.007*"outback" + 0.007*"miss"
Topic: 5 Word: 0.017*"charg" + 0.016*"polic" + 0.016*"murder" + 0.013*"court" + 0.

# Classification of the topics

# Performance evaluation by classifiying sampl edocument using LDA Bag of Words model

In [24]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [25]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.41517260670661926	 
Topic: 0.029*"govern" + 0.019*"health" + 0.019*"school" + 0.017*"help" + 0.017*"chang" + 0.015*"feder" + 0.014*"coronavirus" + 0.013*"indigen" + 0.012*"state" + 0.012*"fund"

Score: 0.34662461280822754	 
Topic: 0.025*"call" + 0.025*"tasmania" + 0.020*"rise" + 0.019*"victorian" + 0.017*"morrison" + 0.016*"tasmanian" + 0.015*"million" + 0.015*"farm" + 0.011*"program" + 0.011*"town"

Score: 0.15067148208618164	 
Topic: 0.071*"australia" + 0.045*"trump" + 0.025*"donald" + 0.016*"border" + 0.015*"elect" + 0.015*"busi" + 0.015*"peopl" + 0.014*"accus" + 0.013*"say" + 0.012*"scott"

Score: 0.012506254948675632	 
Topic: 0.054*"coronavirus" + 0.035*"covid" + 0.024*"live" + 0.021*"nation" + 0.020*"coast" + 0.016*"restrict" + 0.014*"water" + 0.013*"gold" + 0.011*"plan" + 0.010*"park"

Score: 0.012504437938332558	 
Topic: 0.037*"year" + 0.031*"melbourn" + 0.022*"open" + 0.021*"canberra" + 0.017*"jail" + 0.015*"work" + 0.014*"high" + 0.014*"life" + 0.013*"interview" + 0

# Performance evaluation by classifying sample document using LDA TF-IDF model

In [26]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7302262187004089	 
Topic: 0.014*"coronavirus" + 0.009*"health" + 0.007*"restrict" + 0.007*"fund" + 0.006*"care" + 0.006*"stori" + 0.006*"covid" + 0.006*"budget" + 0.005*"queensland" + 0.005*"plan"

Score: 0.16972662508487701	 
Topic: 0.019*"news" + 0.018*"market" + 0.015*"donald" + 0.013*"rural" + 0.012*"coronavirus" + 0.012*"drum" + 0.009*"price" + 0.008*"tuesday" + 0.008*"share" + 0.007*"wednesday"

Score: 0.012508132494986057	 
Topic: 0.014*"govern" + 0.011*"elect" + 0.009*"coronavirus" + 0.007*"china" + 0.007*"say" + 0.006*"south" + 0.006*"australia" + 0.006*"korea" + 0.006*"liber" + 0.006*"labor"

Score: 0.012505987659096718	 
Topic: 0.020*"countri" + 0.015*"hour" + 0.011*"weather" + 0.009*"michael" + 0.009*"turnbul" + 0.009*"extend" + 0.007*"brief" + 0.007*"disabl" + 0.007*"octob" + 0.005*"america"

Score: 0.012505955062806606	 
Topic: 0.020*"live" + 0.011*"search" + 0.011*"covid" + 0.010*"andrew" + 0.009*"updat" + 0.009*"financ" + 0.009*"coronavirus" + 0.009*"peter" + 

# Testing model on unseen document

In [27]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.34983351826667786	 Topic: 0.026*"death" + 0.025*"charg" + 0.025*"case" + 0.024*"court" + 0.020*"polic"
Score: 0.3451051414012909	 Topic: 0.022*"news" + 0.020*"market" + 0.018*"world" + 0.017*"women" + 0.015*"australian"
Score: 0.18833015859127045	 Topic: 0.041*"queensland" + 0.033*"victoria" + 0.022*"bushfir" + 0.021*"hous" + 0.014*"time"
Score: 0.016682103276252747	 Topic: 0.030*"china" + 0.025*"test" + 0.020*"south" + 0.015*"coronavirus" + 0.013*"north"
Score: 0.016676826402544975	 Topic: 0.025*"call" + 0.025*"tasmania" + 0.020*"rise" + 0.019*"victorian" + 0.017*"morrison"
Score: 0.016676053404808044	 Topic: 0.029*"govern" + 0.019*"health" + 0.019*"school" + 0.017*"help" + 0.017*"chang"
Score: 0.01667594164609909	 Topic: 0.054*"coronavirus" + 0.035*"covid" + 0.024*"live" + 0.021*"nation" + 0.020*"coast"
Score: 0.016673417761921883	 Topic: 0.038*"sydney" + 0.025*"polic" + 0.021*"crash" + 0.020*"adelaid" + 0.019*"die"
Score: 0.016673417761921883	 Topic: 0.037*"year" + 0.031*"m

In [28]:
unseen_document_1 = "A hacker who stole just over $600m (£433m) worth of cryptocurrency has now returned most of the stolen assets.On Thursday, Poly Network confirmed on Twitter that $268m worth of Ether tokens had now been recovered.Over the last 24 hours, the hacker has returned $342m worth of tokens relating to three crypto-currencies to the firm.The individual also posted several pages of notes to the blockchain, disclosing why they hacked the firm and the offers Poly Network made to them.In a twist that's worrying some cyber-security experts, the hacker claims the firm offered to pay $500,000 if they returned the stolen assets, as well as a promise of immunity from prosecution."
bow_vector = dictionary.doc2bow(preprocess(unseen_document_1))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))




Score: 0.2350327968597412	 Topic: 0.029*"govern" + 0.019*"health" + 0.019*"school" + 0.017*"help" + 0.017*"chang"
Score: 0.19003261625766754	 Topic: 0.054*"coronavirus" + 0.035*"covid" + 0.024*"live" + 0.021*"nation" + 0.020*"coast"
Score: 0.15700757503509521	 Topic: 0.026*"death" + 0.025*"charg" + 0.025*"case" + 0.024*"court" + 0.020*"polic"
Score: 0.09534835815429688	 Topic: 0.022*"news" + 0.020*"market" + 0.018*"world" + 0.017*"women" + 0.015*"australian"
Score: 0.09534744918346405	 Topic: 0.037*"year" + 0.031*"melbourn" + 0.022*"open" + 0.021*"canberra" + 0.017*"jail"
Score: 0.09145008772611618	 Topic: 0.025*"call" + 0.025*"tasmania" + 0.020*"rise" + 0.019*"victorian" + 0.017*"morrison"
Score: 0.05042131245136261	 Topic: 0.030*"china" + 0.025*"test" + 0.020*"south" + 0.015*"coronavirus" + 0.013*"north"
Score: 0.030918247997760773	 Topic: 0.071*"australia" + 0.045*"trump" + 0.025*"donald" + 0.016*"border" + 0.015*"elect"
Score: 0.02890424244105816	 Topic: 0.041*"queensland" + 0.033*